# AhmadReza Sezavar, 810198173

In [1]:
from proposed import *

In [2]:
def calculate_time(nodes):
    """compute the total time"""
    time_ = 0
    for node in nodes:
        """ for calculate time, we have to consider to exit node and left the others """
        if len(node.successors) == 0:
            """ find finish time on local core or recv channel """
            time_ = max(node.ft_l, node.ft_wr)
    return time_

def calculate_energy(nodes, core_cloud_power=[1, 2, 4, 0.5]):
    total_energy = 0
    for node in nodes:
        if node.on_device == True:
            """ for calculate energy we use the Eqs in the paper energy = Sigma(Power * Time Consumption) """
            node_energy =  core_cloud_power[node.assignment] * node.speed_on_core[node.assignment]
            total_energy += node_energy
        if node.on_device == False:
            node_energy =  core_cloud_power[3] * node.speed_on_cloud[0]
            total_energy += node_energy
    return total_energy

In [3]:
""" Create Tasks as nodes"""
task10= Task(id=10, successors=[], speed_on_core=[7, 4, 2])
task9 = Task(id=9,  successors=[task10], speed_on_core=[5, 3, 2])
task8 = Task(id=8,  successors=[task10], speed_on_core=[6, 4, 2])
task7 = Task(id=7,  successors=[task10], speed_on_core=[8, 5, 3])
task6 = Task(id=6,  successors=[task8], speed_on_core=[7, 6, 4])
task5 = Task(id=5,  successors=[task9], speed_on_core=[5, 4, 2])
task4 = Task(id=4,  successors=[task8, task9], speed_on_core=[7, 5, 3])
task3 = Task(id=3,  successors=[task7], speed_on_core=[6, 5, 4])
task2 = Task(id=2,  successors=[task8, task9], speed_on_core=[8, 6, 5])
task1 = Task(id=1,  successors=[task2, task3, task4, task5, task6], speed_on_core=[9, 7, 5])
""" define predecessors of each node"""
task1.predecessors = []
task2.predecessors = [task1]
task3.predecessors = [task1]
task4.predecessors = [task1]
task5.predecessors = [task1]
task6.predecessors = [task1]
task7.predecessors = [task3]
task8.predecessors = [task2, task4, task6]
task9.predecessors = [task2, task4, task5]
task10.predecessors = [task7, task8, task9]
tasks_ = [task10, task9, task8, task7, task6, task5, task4, task3, task2, task1]

""" paper said task2 is sch on cloud at initial"""
task2.on_device = False

""" set ready time of entry task =0 at initial"""
task1.ready_time = 0

""" Calculate priority score of each node 
    first: calculate w for each node
    second: calculate priority score using w and consider predecessors and successors of each node
"""
# step 1
for node in tasks_:
    # w_i = 0
    if node.on_device == True:
        node.w_i = sum(node.speed_on_core) / len(node.speed_on_core)
    else:
        node.w_i = 5
# step 2
for node in tasks_:
    pr_val = node.w_i
    if len(node.successors) == 0:
        node.pr_val = pr_val
        continue
    successor_score = max([i.pr_val for i in node.successors])

    node.pr_val = pr_val + successor_score
tasks_ = sorted(tasks_, key=lambda node: node.pr_val, reverse=True)
print("Priority Order")
for node in tasks_:
    print('Task: ' , node.id, '     Priority Score: ' , node.pr_val)

""" primary assignment, calculate ready time and finish time """
start_locally = [0, 0, 0]   # related to core1, core2, core3
start_cloud = [0, 0, 0]     # related to ws, c, wr
""" sequence of tasks on each cores and cloud"""
sch_on_core1 = []
sch_on_core2 = []
sch_on_core3 = []
sch_on_cloud = []

""" a loop over each node for find Ready time and finish time"""
for i, node in enumerate(tasks_):
    """ if node is local and have no predecessors like task1 -> ready time =0"""
    if node.on_device == True:
        if len(node.predecessors) == 0:
            node.ready_time = 0
        else:
            for p in node.predecessors:
                """ if node is local and have predecessors use eq.3 for find ready time"""
                p_ft = max(p.ft_l, p.ft_wr)
                if p_ft > node.ready_time:
                    node.ready_time = p_ft

        """ calculate finish time of task i on each local core"""
        finish_core1 = max(start_locally[0], node.ready_time) + node.speed_on_core[0]
        finish_core2 = max(start_locally[1], node.ready_time) + node.speed_on_core[1]
        finish_core3 = max(start_locally[2], node.ready_time) + node.speed_on_core[2]
        """ then select the fastest core """
        assignment_id = 0
        assignment_finish_time = finish_core1
        if assignment_finish_time > finish_core2:
            assignment_finish_time = finish_core2
            assignment_id = 1
        if assignment_finish_time > finish_core3:
            assignment_finish_time = finish_core3
            assignment_id = 2
        node.assignment = assignment_id
        """ find the finish time of task i on selected core"""
        node.ft_l = assignment_finish_time
        """ assign start time of task"""
        node.start_time[assignment_id] = max(start_locally[assignment_id], node.ready_time)
        start_locally[assignment_id] = node.ft_l

        """ update the sequence of each core"""
        if node.assignment == 0:
            sch_on_core1.append(node.id)
        if node.assignment == 1:
            sch_on_core2.append(node.id)
        if node.assignment == 2:
            sch_on_core3.append(node.id)

        print("ID of Task ->{}, sch on core ->{}, ready time -> {}, start_time on core -> {}".format(node.id, node.assignment + 1, node.ready_time, node.start_time[node.assignment]) , ' sch line: ' , start_locally )

    """ if node is a cloud task, we use eq.4 for find the ready time of WS"""
    if node.on_device == False:
        for p in node.predecessors:
            p_ws = max(p.ft_l, p.ft_ws)
            if p_ws > node.rt_ws:
                node.rt_ws = p_ws
        ws_finish = max(start_cloud[0], node.rt_ws) + node.speed_on_cloud[0]
        node.ft_ws = ws_finish
        """ then use eq.5 for find the ready time of C"""
        p_max_ft_c = 0
        for p in node.predecessors:
            if p.ft_c > p_max_ft_c:
                p_max_ft_c = p.ft_c
        node.rt_c = max(node.ft_ws, p_max_ft_c)
        c_finish = max(start_cloud[1], node.rt_c) + node.speed_on_cloud[1]
        node.ft_c = c_finish
        """ then use eq.6 for find the ready time of WR"""
        node.rt_wr = node.ft_c
        wr_finish = max(start_cloud[2], node.rt_wr) + node.speed_on_cloud[2]
        node.ft_wr = wr_finish
        """ assign id and finish time and start time"""
        node.assignment = 3
        node.start_time[3] = max(start_cloud[0], node.rt_ws)
        start_cloud[0] = ws_finish
        start_cloud[1] = c_finish
        start_cloud[2] = wr_finish
        sch_on_cloud.append(node.id)
        print( "ID of Task ->{}, assigenment ->{}, ws ready time -> {}, c ready time -> {}, wr ready time -> {}, start time -> {}".format(node.id, node.assignment + 1, node.rt_ws, node.rt_c, node.rt_wr, node.start_time[3]) , ' sch line: ' , start_locally)

print("Total Time   ===> ", calculate_time(tasks_))
print("Total Energy ===> ", calculate_energy(tasks_, [1, 2, 4, 0.5]))
print('schedule on core 1: ', [i for i in sch_on_core1])
print('schedule on core 2: ', [i for i in sch_on_core2])
print('schedule on core 3: ', [i for i in sch_on_core3])
print('schedule on cloud: ', [i for i in sch_on_cloud])
sequence = [sch_on_core1, sch_on_core2, sch_on_core3, sch_on_cloud]

""" end of primary assignment, ready time and finish time"""
time_in = calculate_time(tasks_)
energy_in = calculate_energy(tasks_, [1, 2, 4, 0.5])
print("Time and Energy (at initial): ", time_in, energy_in)


""" Outer Loop """
len_of_performance = 0
while len_of_performance < 10:
    # One outer loop
    print("\n\n\t\t\t iteration: ", len_of_performance)
    print("=" * 50)
    """ first calculate time and energy at the top of the loop"""
    time_in = calculate_time(tasks_)
    energy_in = calculate_energy(tasks_, [1, 2, 4, 0.5])
    print("Time and Energy (at initial): ", time_in, energy_in)

    """ if a node is a cloud task: all its elements in choice of migration will be [1,1,1,1] 
        else if a node sch on first local -> [1,0,0,0] and etc
        note that we have 4 resource so each element has 4 numbers.
    """
    """ initialization of nested list """
    choice_of_migration = []
    for i in range(len(tasks_)):
        choice_of_migration.append([])

    """ fill the choice_of_migration nested list"""
    for i in range(len(tasks_)):
        """ if task sch on cloud"""
        if tasks_[i].assignment == 3:
            """ id is in range (1,10) so new tmp id must be in range (0,9)"""
            new_tmp_id = tasks_[i].id - 1
            new_tmp_val = [1] * 4         # put 1 because we don't transfer from cloud to local

            choice_of_migration[new_tmp_id] = new_tmp_val      # [1,1,1,1]
        else:
            """ if task sch on cloud"""
            new_tmp_id = tasks_[i].id - 1
            new_tmp_val = [0] * 4
            new_tmp_val[tasks_[i].assignment] = 1

            choice_of_migration[new_tmp_id] = new_tmp_val      # [1,0,0,0] or [0,1,0,0] or ...
    """ show output """
    print("Migration ")
    # print('choice_of_migration: ', choice_of_migration)

    """ create a temporary list for find the best energy and time """
    tmp_output = []
    tmp1 = []
    for i in range(len(tasks_)):
        tmp1 = []
        for j in range(4):
            tmp1.append((-1, -1))
        tmp_output.append(tmp1)

    """ now we check all of the migration choices """
    for node_ in range(len(choice_of_migration)):
        mig_choice = choice_of_migration[node_]  # [0, 0, 1, 0] , ...
        """ for k th resource """
        for k in range(len(mig_choice)):
            """ if cloud or been sch before"""
            if mig_choice[k] == 1:
                continue
            """ take a copy of original seq and nodes"""
            sequence_tmp = deepcopy(sequence)               # [[4], [6, 8], [1, 3, 5, 7, 9, 10]
            nodes_tmp = deepcopy(tasks_)                    # each node

            """  
            Generate new sequence, tar_id = node_+1 
            """

            """ create a dictionary that keys are node id and values are index :{key-node.id: value-index in nodes}"""
            id_dict = {}
            new_id = 0
            """ define id from range(0,9) and tar node"""
            for each in nodes_tmp:
                id_dict[each.id] = new_id
                new_id += 1
                if each.id == node_ + 1:  # target node
                    target_node = each

            """ define ready time """
            if target_node.on_device == True:
                target_node_rt = target_node.ready_time
            if target_node.on_device == False:
                target_node_rt = target_node.rt_ws
            sequence_tmp[target_node.assignment].remove(target_node.id)
            """ using eq.19 for update S new"""
            s_new = sequence_tmp[k]
            s_new_tmpfinal = []
            """ update sequence"""
            flag = False
            for id_of_node in s_new:
                """ only consider nodes in target core and don't change the others"""
                each = nodes_tmp[id_dict[id_of_node]]
                if each.start_time[k] < target_node_rt:
                    s_new_tmpfinal.append(each.id)
                if each.start_time[k] >= target_node_rt and flag == False:
                    s_new_tmpfinal.append(target_node.id)
                    flag = True
                if each.start_time[k] >= target_node_rt and flag == True:
                    s_new_tmpfinal.append(each.id)
            if flag == False:
                s_new_tmpfinal.append(target_node.id)
            sequence_tmp[k] = s_new_tmpfinal
            target_node.assignment = k
            if k == 3:
                target_node.on_device = False
            else:
                target_node.on_device = True

            """ Now new sequence is generated, we pass it to Kernel algorithm"""

            """ after generate a new sequence, run the kernel algorithm"""
            start_local = [0, 0, 0]
            start_cloud = [0, 0, 0]
            """ define ready1 and ready2 as paper said"""
            ready1 = [-1] * len(
                nodes_tmp)  # [-1s] at start, ready1[i] is for node.id==i , [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
            ready2 = [-1] * len(nodes_tmp)  # [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
            ready1[nodes_tmp[0].id - 1] = 0  # id start from 1 , ready1: [0, -1, -1, -1, -1, -1, -1, -1, -1, -1]

            for each_ in sequence_tmp:
                if len(each_) > 0:
                    """ 
                    ready2_i=0 if all tasks before task vi in the same seq Sk have already been scheduled.
                    Example: if seq: [[1, 4], [6, 8], [3, 5, 7, 9, 10], [2]], ready2 of 1,6,3,2 must be 0
                    because we dont have any tasks before them.
                    so each seq[0]: 1, 6, 3, 2 and 
                    after run below code the final ready2: [0, 0, 0, -1, -1, 0, -1, -1, -1, -1]"""
                    ready2[each_[0] - 1] = 0

            """put id from 0 to 9 to all nodes"""
            dict_id = {}  # {key-node.id: value-index in nodes}
            new_id = 0
            for each in nodes_tmp:
                dict_id[each.id] = new_id
                each.ready_time = -1  # local ready time
                each.rt_ws = -1  # cloud ready time
                each.rt_c = -1
                each.rt_wr = -1
                new_id += 1
            """ define LIFO stack as paper said"""
            stack = []
            stack.append(nodes_tmp[0])  # init by ready1_i == ready2_i == 0

            while len(stack) != 0:  # not empty
                last_input = stack.pop()
                last_input.check_kernel = "kernel_done"  # means is scheduled
                """ first, calculate last_input local ready time """
                if last_input.on_device == True:
                    if len(last_input.predecessors) == 0:
                        last_input.ready_time = 0
                    else:
                        for p in last_input.predecessors:
                            p_ft = max(p.ft_l, p.ft_wr)
                            if p_ft > last_input.ready_time:
                                last_input.ready_time = p_ft

                """ schedule on the corresponding core """
                if last_input.assignment == 0:
                    last_input.start_time = [-1, -1, -1, -1]
                    last_input.start_time[0] = max(start_local[0], last_input.ready_time)
                    last_input.ft_l = last_input.start_time[0] + last_input.speed_on_core[0]
                    last_input.ft_ws = -1
                    last_input.ft_c = -1
                    last_input.ft_wr = -1
                    start_local[0] = last_input.ft_l
                """ local 1"""
                if last_input.assignment == 1:
                    last_input.start_time = [-1, -1, -1, -1]
                    last_input.start_time[1] = max(start_local[1], last_input.ready_time)
                    last_input.ft_l = last_input.start_time[1] + last_input.speed_on_core[1]
                    last_input.ft_ws = -1
                    last_input.ft_c = -1
                    last_input.ft_wr = -1
                    start_local[1] = last_input.ft_l
                """ local 2"""
                if last_input.assignment == 2:
                    last_input.start_time = [-1, -1, -1, -1]
                    last_input.start_time[2] = max(start_local[2], last_input.ready_time)
                    last_input.ft_l = last_input.start_time[2] + last_input.speed_on_core[2]
                    last_input.ft_ws = -1
                    last_input.ft_c = -1
                    last_input.ft_wr = -1
                    start_local[2] = last_input.ft_l
                """ cloud and first Sending"""
                if last_input.assignment == 3:
                    if len(last_input.predecessors) == 0:
                        last_input.rt_ws = 0
                    else:
                        for p in last_input.predecessors:
                            p_ws = max(p.ft_l, p.ft_ws)
                            if p_ws > last_input.rt_ws:
                                last_input.rt_ws = p_ws
                    last_input.ft_ws = max(start_cloud[0], last_input.rt_ws) + last_input.speed_on_cloud[0]
                    last_input.start_time = [-1, -1, -1, -1]
                    last_input.start_time[3] = max(start_cloud[0], last_input.rt_ws)
                    start_cloud[0] = last_input.ft_ws
                    """ cloud C"""
                    p_max_ft_c = 0
                    for p in last_input.predecessors:
                        if p.ft_c > p_max_ft_c:
                            p_max_ft_c = p.ft_c
                    last_input.rt_c = max(last_input.ft_ws, p_max_ft_c)
                    last_input.ft_c = max(start_cloud[1], last_input.rt_c) + last_input.speed_on_cloud[1]
                    start_cloud[1] = last_input.ft_c
                    """ cloud WR"""
                    last_input.rt_wr = last_input.ft_c
                    last_input.ft_wr = max(start_cloud[2], last_input.rt_wr) + last_input.speed_on_cloud[2]
                    last_input.ft_l = -1
                    start_cloud[2] = last_input.ft_wr

                """ ready1 and ready 2 updated"""
                corresponding_seq = sequence_tmp[last_input.assignment]
                v_i_index = corresponding_seq.index(last_input.id)  # position of last_input in seq list
                if v_i_index != len(corresponding_seq) - 1:  # not be the last one
                    next_node_id = corresponding_seq[v_i_index + 1]
                else:
                    next_node_id = -1

                for each in nodes_tmp:
                    flag = 0
                    for p in each.predecessors:
                        """if node's parent scheduled before ready1 of node=0"""
                        if p.check_kernel != "kernel_done":
                            flag += 1
                        ready1[each.id - 1] = flag
                    if each.id == next_node_id:
                        ready2[each.id - 1] = 0

                for each in nodes_tmp:
                    """ adding node"""
                    if (ready1[each.id - 1] == 0) and (ready2[each.id - 1] == 0) and (
                            each.check_kernel != "kernel_done") and (each not in stack):
                        stack.append(each)

            for node in nodes_tmp:
                node.check_kernel = None

            """ Finish Kernel algo """
            """ after run the kernel algorithm, calculate time and energy for find the best case later"""
            time_now = calculate_time(nodes_tmp)
            energy_now = calculate_energy(nodes_tmp)
            """ delete copy"""
            del nodes_tmp
            del sequence_tmp
            """ store energy and time for finc the best later"""
            tmp_output[node_][k] = (time_now, energy_now)


    final_n = -1
    final_resource_k = -1
    final_time = time_in
    final_energy = energy_in
    final_delta_ratio_ET = -1
    """ for find the best migration and sch case
    consider all nodes(tasks) and migration choice and current schedulability"""
    for i in range(len(tmp_output)):        # 10 tasks
        for j in range(len(tmp_output[i])): # 4 mig choice
            """ find current value if not changed and more than constrained must be checked again"""
            current = tmp_output[i][j]
            if current == (-1, -1):
                continue
            if current[0] > 27:
                continue

            """ find the ratio and take the best"""
            delta_ratio_ET = (final_energy - current[1]) / abs(current[0] - final_time + 0.00005)
            if delta_ratio_ET > final_delta_ratio_ET:
                final_delta_ratio_ET = delta_ratio_ET
                final_n = i
                final_resource_k = j

    if final_n == -1 and final_resource_k == -1:
        break
    final_n += 1
    final_resource_k += 1
    final_time, final_energy = tmp_output[final_n - 1][final_resource_k - 1]
    print("Migration task-> {},  k-> {} ".format(final_n, final_resource_k))
    print("Updating")

    """ Generate new sequence """
    """ create a dictionary that keys are node id and values are index :{key-node.id: value-index in nodes}"""
    id_dict = {}
    new_id = 0
    """ define id from range(0,9) and tar node"""
    for each in tasks_:
        id_dict[each.id] = new_id
        new_id += 1
        if each.id == final_n:  # target
            target_node = each

    """ define ready time """
    if target_node.on_device == True:
        target_node_rt = target_node.ready_time
    if target_node.on_device == False:
        target_node_rt = target_node.rt_ws
    sequence[target_node.assignment].remove(target_node.id)
    """ using eq.19 for update S new"""
    s_new = sequence[final_resource_k - 1]
    s_new_tmpfinal = []
    """ update sequence"""
    flag = False
    for id_of_node in s_new:
        """ only consider nodes in target core and don't change the others"""
        each = tasks_[id_dict[id_of_node]]
        if each.start_time[final_resource_k - 1] < target_node_rt:
            s_new_tmpfinal.append(each.id)
        if each.start_time[final_resource_k - 1] >= target_node_rt and flag == False:
            s_new_tmpfinal.append(target_node.id)
            flag = True
        if each.start_time[final_resource_k - 1] >= target_node_rt and flag == True:
            s_new_tmpfinal.append(each.id)
    if flag == False:
        s_new_tmpfinal.append(target_node.id)
    sequence[final_resource_k - 1] = s_new_tmpfinal
    target_node.assignment = final_resource_k - 1
    if final_resource_k - 1 == 3:
        target_node.on_device = False
    else:
        target_node.on_device = True

    """
    new seq generated, pass it to the Kernel algo
    """
    start_local = [0, 0, 0]
    start_cloud = [0, 0, 0]
    """ define ready1 and ready2 as paper said"""
    ready1 = [-1] * len(tasks_)  # [-1s] at start, ready1[i] is for node.id==i , [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
    ready2 = [-1] * len(tasks_)  # [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
    ready1[tasks_[0].id - 1] = 0  # id start from 1 , ready1: [0, -1, -1, -1, -1, -1, -1, -1, -1, -1]

    for each_ in sequence:
        if len(each_) > 0:
            """ 
            ready2_i=0 if all tasks before task vi in the same seq Sk have already been scheduled.
            Example: if seq: [[1, 4], [6, 8], [3, 5, 7, 9, 10], [2]], ready2 of 1,6,3,2 must be 0
            because we dont have any tasks before them.
            so each seq[0]: 1, 6, 3, 2 and 
            after run below code the final ready2: [0, 0, 0, -1, -1, 0, -1, -1, -1, -1]"""
            ready2[each_[0] - 1] = 0

    """put id from 0 to 9 to all nodes"""
    dict_id = {}  # {key-node.id: value-index in nodes}
    new_id = 0
    for each in tasks_:
        dict_id[each.id] = new_id
        each.ready_time = -1  # local ready time
        each.rt_ws = -1  # cloud ready time
        each.rt_c = -1
        each.rt_wr = -1
        new_id += 1
    """ define LIFO stack as paper said"""
    stack = []
    stack.append(tasks_[0])  # init by ready1_i == ready2_i == 0

    while len(stack) != 0:  # not empty
        last_input = stack.pop()
        last_input.check_kernel = "kernel_done"  # means is scheduled
        """ first, calculate last_input local ready time """
        if last_input.on_device == True:
            if len(last_input.predecessors) == 0:
                last_input.ready_time = 0
            else:
                for p in last_input.predecessors:
                    p_ft = max(p.ft_l, p.ft_wr)
                    if p_ft > last_input.ready_time:
                        last_input.ready_time = p_ft

        """ schedule on the corresponding core """
        if last_input.assignment == 0:
            last_input.start_time = [-1, -1, -1, -1]
            last_input.start_time[0] = max(start_local[0], last_input.ready_time)
            last_input.ft_l = last_input.start_time[0] + last_input.speed_on_core[0]
            last_input.ft_ws = -1
            last_input.ft_c = -1
            last_input.ft_wr = -1
            start_local[0] = last_input.ft_l
        """ local 1"""
        if last_input.assignment == 1:
            last_input.start_time = [-1, -1, -1, -1]
            last_input.start_time[1] = max(start_local[1], last_input.ready_time)
            last_input.ft_l = last_input.start_time[1] + last_input.speed_on_core[1]
            last_input.ft_ws = -1
            last_input.ft_c = -1
            last_input.ft_wr = -1
            start_local[1] = last_input.ft_l
        """ local 2"""
        if last_input.assignment == 2:
            last_input.start_time = [-1, -1, -1, -1]
            last_input.start_time[2] = max(start_local[2], last_input.ready_time)
            last_input.ft_l = last_input.start_time[2] + last_input.speed_on_core[2]
            last_input.ft_ws = -1
            last_input.ft_c = -1
            last_input.ft_wr = -1
            start_local[2] = last_input.ft_l
        """ cloud and first Sending"""
        if last_input.assignment == 3:
            if len(last_input.predecessors) == 0:
                last_input.rt_ws = 0
            else:
                for p in last_input.predecessors:
                    p_ws = max(p.ft_l, p.ft_ws)
                    if p_ws > last_input.rt_ws:
                        last_input.rt_ws = p_ws
            last_input.ft_ws = max(start_cloud[0], last_input.rt_ws) + last_input.speed_on_cloud[0]
            last_input.start_time = [-1, -1, -1, -1]
            last_input.start_time[3] = max(start_cloud[0], last_input.rt_ws)
            start_cloud[0] = last_input.ft_ws
            """ cloud C"""
            p_max_ft_c = 0
            for p in last_input.predecessors:
                if p.ft_c > p_max_ft_c:
                    p_max_ft_c = p.ft_c
            last_input.rt_c = max(last_input.ft_ws, p_max_ft_c)
            last_input.ft_c = max(start_cloud[1], last_input.rt_c) + last_input.speed_on_cloud[1]
            start_cloud[1] = last_input.ft_c
            """ cloud WR"""
            last_input.rt_wr = last_input.ft_c
            last_input.ft_wr = max(start_cloud[2], last_input.rt_wr) + last_input.speed_on_cloud[2]
            last_input.ft_l = -1
            start_cloud[2] = last_input.ft_wr

        """ ready1 and ready 2 updated"""
        corresponding_seq = sequence[last_input.assignment]
        v_i_index = corresponding_seq.index(last_input.id)  # position of last_input in seq list
        if v_i_index != len(corresponding_seq) - 1:  # not be the last one
            next_node_id = corresponding_seq[v_i_index + 1]
        else:
            next_node_id = -1

        for each in tasks_:
            flag = 0
            for p in each.predecessors:
                """if node's parent scheduled before ready1 of node=0"""
                if p.check_kernel != "kernel_done":
                    flag += 1
                ready1[each.id - 1] = flag
            if each.id == next_node_id:
                ready2[each.id - 1] = 0

        for each in tasks_:
            """ adding node"""
            if (ready1[each.id - 1] == 0) and (ready2[each.id - 1] == 0) and (
                    each.check_kernel != "kernel_done") and (each not in stack):
                stack.append(each)

    for node in tasks_:
        node.check_kernel = None

    """ Kernel finished """

    """ print sch on cores"""
    tmp0 = 0
    for a in sequence:
        if tmp0 != 3:
            print('core {}'.format(tmp0+1), ' : ', [i for i in a])
        else:
            print('cloud : ', [i for i in a])
        tmp0 += 1

    Time_now = calculate_time(tasks_)
    Energy_now = calculate_energy(tasks_, [1, 2, 4, 0.5])
    E_diff = energy_in - Energy_now
    T_diff = abs(Time_now - time_in)
    len_of_performance += 1

    print("Time and Energy: ", Time_now, Energy_now)
    if E_diff <= 1:
        break

for node in tasks_:
    if node.on_device == True:
        print("Task ID: {}, assigenment:{}, ready time: {}, start_time on core: {}".format(node.id, node.assignment + 1, node.ready_time, node.start_time[node.assignment]))
    else:
        print("Task ID:{}, assigenment:{}, ws ready time: {}, c ready time: {}, wr ready time: {}, start time: {}".format(node.id, node.assignment + 1, node.rt_ws, node.rt_c, node.rt_wr, node.start_time[3]))

print("Final schedulability ")
tmp0 = 0
for a in sequence:
    if tmp0 !=3:
        print('core {}'.format(tmp0+1) , ' : ', [i for i in a])
    else:
        print('cloud : ', [i for i in a])
    tmp0 +=1

T_final = calculate_time(tasks_)
E_final = calculate_energy(tasks_, [1, 2, 4, 0.5])
print("Final Time: {}, Final Energy: {}".format(Time_now, Energy_now))

Priority Order
Task:  1      Priority Score:  21.666666666666664
Task:  3      Priority Score:  14.666666666666666
Task:  6      Priority Score:  14.0
Task:  4      Priority Score:  13.333333333333332
Task:  2      Priority Score:  13.333333333333332
Task:  5      Priority Score:  11.333333333333332
Task:  7      Priority Score:  9.666666666666666
Task:  8      Priority Score:  8.333333333333332
Task:  9      Priority Score:  7.666666666666666
Task:  10      Priority Score:  4.333333333333333
ID of Task ->1, sch on core ->3, ready time -> 0, start_time on core -> 0  sch line:  [0, 0, 5]
ID of Task ->3, sch on core ->3, ready time -> 5, start_time on core -> 5  sch line:  [0, 0, 9]
ID of Task ->6, sch on core ->2, ready time -> 5, start_time on core -> 5  sch line:  [0, 11, 9]
ID of Task ->4, sch on core ->1, ready time -> 5, start_time on core -> 5  sch line:  [12, 11, 9]
ID of Task ->2, assigenment ->4, ws ready time -> 5, c ready time -> 8, wr ready time -> 9, start time -> 5  sch li

# Baseline

In [4]:
from baseline import *

In [5]:
def calculate_time(nodes):
    """compute the total time"""
    time_ = 0
    for node in nodes:
        """ for calculate time, we have to consider to exit node and left the others """
        if len(node.successors) == 0:
            """ find finish time on local core or recv channel """
            time_ = node.ft_l
    return time_

def calculate_energy(nodes, core_cloud_power=[1, 2, 4, 0.5]):
    total_energy = 0
    for node in nodes:
        if node.on_device == True:
            """ for calculate energy we use the Eqs in the paper energy = Sigma(Power * Time Consumption) """
            node_energy =  core_cloud_power[node.assignment] * node.speed_on_core[node.assignment]
            total_energy += node_energy

    return total_energy

In [6]:
""" Create Tasks as nodes"""
task10= Task(id=10, successors=[], speed_on_core=[7, 4, 2])
task9 = Task(id=9,  successors=[task10], speed_on_core=[5, 3, 2])
task8 = Task(id=8,  successors=[task10], speed_on_core=[6, 4, 2])
task7 = Task(id=7,  successors=[task10], speed_on_core=[8, 5, 3])
task6 = Task(id=6,  successors=[task8], speed_on_core=[7, 6, 4])
task5 = Task(id=5,  successors=[task9], speed_on_core=[5, 4, 2])
task4 = Task(id=4,  successors=[task8, task9], speed_on_core=[7, 5, 3])
task3 = Task(id=3,  successors=[task7], speed_on_core=[6, 5, 4])
task2 = Task(id=2,  successors=[task8, task9], speed_on_core=[8, 6, 5])
task1 = Task(id=1,  successors=[task2, task3, task4, task5, task6], speed_on_core=[9, 7, 5])
""" define predecessors of each node"""
task1.predecessors = []
task2.predecessors = [task1]
task3.predecessors = [task1]
task4.predecessors = [task1]
task5.predecessors = [task1]
task6.predecessors = [task1]
task7.predecessors = [task3]
task8.predecessors = [task2, task4, task6]
task9.predecessors = [task2, task4, task5]
task10.predecessors = [task7, task8, task9]
tasks_ = [task10, task9, task8, task7, task6, task5, task4, task3, task2, task1]



""" set ready time of entry task =0 at initial"""
task1.ready_time = 0

""" Calculate priority score of each node 
    first: calculate w for each node
    second: calculate priority score using w and consider predecessors and successors of each node
"""
# step 1
for node in tasks_:
    # w_i = 0
    node.w_i = sum(node.speed_on_core) / len(node.speed_on_core)

# step 2
for node in tasks_:
    pr_val = node.w_i
    if len(node.successors) == 0:
        node.pr_val = pr_val
        continue
    successor_score = max([i.pr_val for i in node.successors])

    node.pr_val = pr_val + successor_score
tasks_ = sorted(tasks_, key=lambda node: node.pr_val, reverse=True)
print("Priority Order")
for node in tasks_:
    print('Task: ' , node.id, '     Priority Score: ' , node.pr_val)

""" primary assignment, calculate ready time and finish time """
start_locally = [0, 0, 0]   # related to core1, core2, core3
""" sequence of tasks on each cores and cloud"""
sch_on_core1 = []
sch_on_core2 = []
sch_on_core3 = []

""" a loop over each node for find Ready time and finish time"""
for i, node in enumerate(tasks_):
    """ if node is local and have no predecessors like task1 -> ready time =0"""
    if node.on_device == True:
        if len(node.predecessors) == 0:
            node.ready_time = 0
        else:
            for p in node.predecessors:
                """ if node is local and have predecessors use eq.3 for find ready time"""
                p_ft = p.ft_l
                if p_ft > node.ready_time:
                    node.ready_time = p_ft

        """ calculate finish time of task i on each local core"""
        finish_core1 = max(start_locally[0], node.ready_time) + node.speed_on_core[0]
        finish_core2 = max(start_locally[1], node.ready_time) + node.speed_on_core[1]
        finish_core3 = max(start_locally[2], node.ready_time) + node.speed_on_core[2]
        """ then select the fastest core """
        assignment_id = 0
        assignment_finish_time = finish_core1
        if assignment_finish_time > finish_core2:
            assignment_finish_time = finish_core2
            assignment_id = 1
        if assignment_finish_time > finish_core3:
            assignment_finish_time = finish_core3
            assignment_id = 2
        node.assignment = assignment_id
        """ find the finish time of task i on selected core"""
        node.ft_l = assignment_finish_time
        """ assign start time of task"""
        node.start_time[assignment_id] = max(start_locally[assignment_id], node.ready_time)
        start_locally[assignment_id] = node.ft_l

        """ update the sequence of each core"""
        if node.assignment == 0:
            sch_on_core1.append(node.id)
        if node.assignment == 1:
            sch_on_core2.append(node.id)
        if node.assignment == 2:
            sch_on_core3.append(node.id)

        print("ID of Task ->{}, sch on core ->{}, ready time -> {}, start_time on core -> {}".format(node.id, node.assignment + 1, node.ready_time, node.start_time[node.assignment]) , ' sch line: ' , start_locally )


print("Total Time   ===> ", calculate_time(tasks_))
print("Total Energy ===> ", calculate_energy(tasks_, [1, 2, 4, 0.5]))
print('schedule on core 1: ', [i for i in sch_on_core1])
print('schedule on core 2: ', [i for i in sch_on_core2])
print('schedule on core 3: ', [i for i in sch_on_core3])
sequence = [sch_on_core1, sch_on_core2, sch_on_core3]

""" end of primary assignment, ready time and finish time"""
time_in = calculate_time(tasks_)
energy_in = calculate_energy(tasks_, [1, 2, 4, 0.5])
print("Time and Energy (at initial): ", time_in, energy_in)


""" Outer Loop """
len_of_performance = 0
while len_of_performance < 10:
    # One outer loop
    print("\n\n\t\t\t iteration: ", len_of_performance)
    print("=" * 50)
    """ first calculate time and energy at the top of the loop"""
    time_in = calculate_time(tasks_)
    energy_in = calculate_energy(tasks_, [1, 2, 4, 0.5])
    print("Time and Energy (at initial): ", time_in, energy_in)

    """ if a node is a cloud task: all its elements in choice of migration will be [1,1,1,1] 
        else if a node sch on first local -> [1,0,0,0] and etc
        note that we have 4 resource so each element has 4 numbers.
    """
    """ initialization of nested list """
    choice_of_migration = []
    for i in range(len(tasks_)):
        choice_of_migration.append([])

    """ fill the choice_of_migration nested list"""
    for i in range(len(tasks_)):
        """ if task sch on cloud"""

        """ if task sch on cloud"""
        new_tmp_id = tasks_[i].id - 1
        new_tmp_val = [0] * 3
        new_tmp_val[tasks_[i].assignment] = 1

        choice_of_migration[new_tmp_id] = new_tmp_val      # [1,0,0,0] or [0,1,0,0] or ...
    """ show output """
    print("Migration ")
    # print('choice_of_migration: ', choice_of_migration)

    """ create a temporary list for find the best energy and time """
    tmp_output = []
    tmp1 = []
    for i in range(len(tasks_)):
        tmp1 = []
        for j in range(3):
            tmp1.append((-1, -1))
        tmp_output.append(tmp1)

    """ now we check all of the migration choices """
    for node_ in range(len(choice_of_migration)):
        mig_choice = choice_of_migration[node_]  # [0, 0, 1, 0] , ...
        """ for k th resource """
        for k in range(len(mig_choice)):
            """ if cloud or been sch before"""
            if mig_choice[k] == 1:
                continue
            """ take a copy of original seq and nodes"""
            sequence_tmp = deepcopy(sequence)               # [[4], [6, 8], [1, 3, 5, 7, 9, 10]
            nodes_tmp = deepcopy(tasks_)                    # each node

            """  
            Generate new sequence, tar_id = node_+1 
            """

            """ create a dictionary that keys are node id and values are index :{key-node.id: value-index in nodes}"""
            id_dict = {}
            new_id = 0
            """ define id from range(0,9) and tar node"""
            for each in nodes_tmp:
                id_dict[each.id] = new_id
                new_id += 1
                if each.id == node_ + 1:  # target node
                    target_node = each

            """ define ready time """
            if target_node.on_device == True:
                target_node_rt = target_node.ready_time

            sequence_tmp[target_node.assignment].remove(target_node.id)
            """ using eq.19 for update S new"""
            s_new = sequence_tmp[k]
            s_new_tmpfinal = []
            """ update sequence"""
            flag = False
            for id_of_node in s_new:
                """ only consider nodes in target core and don't change the others"""
                each = nodes_tmp[id_dict[id_of_node]]
                if each.start_time[k] < target_node_rt:
                    s_new_tmpfinal.append(each.id)
                if each.start_time[k] >= target_node_rt and flag == False:
                    s_new_tmpfinal.append(target_node.id)
                    flag = True
                if each.start_time[k] >= target_node_rt and flag == True:
                    s_new_tmpfinal.append(each.id)
            if flag == False:
                s_new_tmpfinal.append(target_node.id)
            sequence_tmp[k] = s_new_tmpfinal
            target_node.assignment = k
            target_node.on_device = True

            """ Now new sequence is generated, we pass it to Kernel algorithm"""

            """ after generate a new sequence, run the kernel algorithm"""
            start_local = [0, 0, 0]
            """ define ready1 and ready2 as paper said"""
            ready1 = [-1] * len(nodes_tmp)  # [-1s] at start, ready1[i] is for node.id==i , [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
            ready2 = [-1] * len(nodes_tmp)  # [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
            ready1[nodes_tmp[0].id - 1] = 0  # id start from 1 , ready1: [0, -1, -1, -1, -1, -1, -1, -1, -1, -1]

            for each_ in sequence_tmp:
                if len(each_) > 0:
                    """ 
                    ready2_i=0 if all tasks before task vi in the same seq Sk have already been scheduled.
                    Example: if seq: [[1, 4], [6, 8], [3, 5, 7, 9, 10], [2]], ready2 of 1,6,3,2 must be 0
                    because we dont have any tasks before them.
                    so each seq[0]: 1, 6, 3, 2 and 
                    after run below code the final ready2: [0, 0, 0, -1, -1, 0, -1, -1, -1, -1]"""
                    ready2[each_[0] - 1] = 0

            """put id from 0 to 9 to all nodes"""
            dict_id = {}  # {key-node.id: value-index in nodes}
            new_id = 0
            for each in nodes_tmp:
                dict_id[each.id] = new_id
                each.ready_time = -1  # local ready time

                new_id += 1
            """ define LIFO stack as paper said"""
            stack = []
            stack.append(nodes_tmp[0])  # init by ready1_i == ready2_i == 0

            while len(stack) != 0:  # not empty
                last_input = stack.pop()
                last_input.check_kernel = "kernel_done"  # means is scheduled
                """ first, calculate last_input local ready time """
                if last_input.on_device == True:
                    if len(last_input.predecessors) == 0:
                        last_input.ready_time = 0
                    else:
                        for p in last_input.predecessors:
                            p_ft = p.ft_l
                            if p_ft > last_input.ready_time:
                                last_input.ready_time = p_ft

                """ schedule on the corresponding core """
                if last_input.assignment == 0:
                    last_input.start_time = [-1, -1, -1, -1]
                    last_input.start_time[0] = max(start_local[0], last_input.ready_time)
                    last_input.ft_l = last_input.start_time[0] + last_input.speed_on_core[0]
                    start_local[0] = last_input.ft_l
                """ local 1"""
                if last_input.assignment == 1:
                    last_input.start_time = [-1, -1, -1, -1]
                    last_input.start_time[1] = max(start_local[1], last_input.ready_time)
                    last_input.ft_l = last_input.start_time[1] + last_input.speed_on_core[1]

                    start_local[1] = last_input.ft_l
                """ local 2"""
                if last_input.assignment == 2:
                    last_input.start_time = [-1, -1, -1, -1]
                    last_input.start_time[2] = max(start_local[2], last_input.ready_time)
                    last_input.ft_l = last_input.start_time[2] + last_input.speed_on_core[2]

                    start_local[2] = last_input.ft_l

                """ ready1 and ready 2 updated"""
                corresponding_seq = sequence_tmp[last_input.assignment]
                v_i_index = corresponding_seq.index(last_input.id)  # position of last_input in seq list
                if v_i_index != len(corresponding_seq) - 1:  # not be the last one
                    next_node_id = corresponding_seq[v_i_index + 1]
                else:
                    next_node_id = -1

                for each in nodes_tmp:
                    flag = 0
                    for p in each.predecessors:
                        """if node's parent scheduled before ready1 of node=0"""
                        if p.check_kernel != "kernel_done":
                            flag += 1
                        ready1[each.id - 1] = flag
                    if each.id == next_node_id:
                        ready2[each.id - 1] = 0

                for each in nodes_tmp:
                    """ adding node"""
                    if (ready1[each.id - 1] == 0) and (ready2[each.id - 1] == 0) and (
                            each.check_kernel != "kernel_done") and (each not in stack):
                        stack.append(each)

            for node in nodes_tmp:
                node.check_kernel = None

            """ Finish Kernel algo """
            """ after run the kernel algorithm, calculate time and energy for find the best case later"""
            time_now = calculate_time(nodes_tmp)
            energy_now = calculate_energy(nodes_tmp)
            """ delete copy"""
            del nodes_tmp
            del sequence_tmp
            """ store energy and time for finc the best later"""
            tmp_output[node_][k] = (time_now, energy_now)


    final_n = -1
    final_resource_k = -1
    final_time = time_in
    final_energy = energy_in
    final_delta_ratio_ET = -1
    """ for find the best migration and sch case
    consider all nodes(tasks) and migration choice and current schedulability"""
    for i in range(len(tmp_output)):        # 10 tasks
        for j in range(len(tmp_output[i])): # 4 mig choice
            """ find current value if not changed and more than constrained must be checked again"""
            current = tmp_output[i][j]
            if current == (-1, -1):
                continue
            if current[0] > 27:
                continue

            """ find the ratio and take the best"""
            delta_ratio_ET = (final_energy - current[1]) / abs(current[0] - final_time + 0.00005)
            if delta_ratio_ET > final_delta_ratio_ET:
                final_delta_ratio_ET = delta_ratio_ET
                final_n = i
                final_resource_k = j

    if final_n == -1 and final_resource_k == -1:
        break
    final_n += 1
    final_resource_k += 1
    final_time, final_energy = tmp_output[final_n - 1][final_resource_k - 1]
    print("Migration task-> {},  k-> {}".format(final_n, final_resource_k))
    print("Updating")

    """ Generate new sequence """
    """ create a dictionary that keys are node id and values are index :{key-node.id: value-index in nodes}"""
    id_dict = {}
    new_id = 0
    """ define id from range(0,9) and tar node"""
    for each in tasks_:
        id_dict[each.id] = new_id
        new_id += 1
        if each.id == final_n:  # target
            target_node = each

    """ define ready time """
    if target_node.on_device == True:
        target_node_rt = target_node.ready_time

    sequence[target_node.assignment].remove(target_node.id)
    """ using eq.19 for update S new"""
    s_new = sequence[final_resource_k - 1]
    s_new_tmpfinal = []
    """ update sequence"""
    flag = False
    for id_of_node in s_new:
        """ only consider nodes in target core and don't change the others"""
        each = tasks_[id_dict[id_of_node]]
        if each.start_time[final_resource_k - 1] < target_node_rt:
            s_new_tmpfinal.append(each.id)
        if each.start_time[final_resource_k - 1] >= target_node_rt and flag == False:
            s_new_tmpfinal.append(target_node.id)
            flag = True
        if each.start_time[final_resource_k - 1] >= target_node_rt and flag == True:
            s_new_tmpfinal.append(each.id)
    if flag == False:
        s_new_tmpfinal.append(target_node.id)
    sequence[final_resource_k - 1] = s_new_tmpfinal
    target_node.assignment = final_resource_k - 1
    target_node.on_device = True

    """
    new seq generated, pass it to the Kernel algo
    """
    start_local = [0, 0, 0]
    """ define ready1 and ready2 as paper said"""
    ready1 = [-1] * len(tasks_)  # [-1s] at start, ready1[i] is for node.id==i , [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
    ready2 = [-1] * len(tasks_)  # [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
    ready1[tasks_[0].id - 1] = 0  # id start from 1 , ready1: [0, -1, -1, -1, -1, -1, -1, -1, -1, -1]

    for each_ in sequence:
        if len(each_) > 0:
            """ 
            ready2_i=0 if all tasks before task vi in the same seq Sk have already been scheduled.
            Example: if seq: [[1, 4], [6, 8], [3, 5, 7, 9, 10], [2]], ready2 of 1,6,3,2 must be 0
            because we dont have any tasks before them.
            so each seq[0]: 1, 6, 3, 2 and 
            after run below code the final ready2: [0, 0, 0, -1, -1, 0, -1, -1, -1, -1]"""
            ready2[each_[0] - 1] = 0

    """put id from 0 to 9 to all nodes"""
    dict_id = {}  # {key-node.id: value-index in nodes}
    new_id = 0
    for each in tasks_:
        dict_id[each.id] = new_id
        each.ready_time = -1  # local ready time

        new_id += 1
    """ define LIFO stack as paper said"""
    stack = []
    stack.append(tasks_[0])  # init by ready1_i == ready2_i == 0

    while len(stack) != 0:  # not empty
        last_input = stack.pop()
        last_input.check_kernel = "kernel_done"  # means is scheduled
        """ first, calculate last_input local ready time """
        if last_input.on_device == True:
            if len(last_input.predecessors) == 0:
                last_input.ready_time = 0
            else:
                for p in last_input.predecessors:
                    p_ft = p.ft_l
                    if p_ft > last_input.ready_time:
                        last_input.ready_time = p_ft

        """ schedule on the corresponding core """
        if last_input.assignment == 0:
            last_input.start_time = [-1, -1, -1, -1]
            last_input.start_time[0] = max(start_local[0], last_input.ready_time)
            last_input.ft_l = last_input.start_time[0] + last_input.speed_on_core[0]
            start_local[0] = last_input.ft_l
        """ local 1"""
        if last_input.assignment == 1:
            last_input.start_time = [-1, -1, -1, -1]
            last_input.start_time[1] = max(start_local[1], last_input.ready_time)
            last_input.ft_l = last_input.start_time[1] + last_input.speed_on_core[1]

            start_local[1] = last_input.ft_l
        """ local 2"""
        if last_input.assignment == 2:
            last_input.start_time = [-1, -1, -1, -1]
            last_input.start_time[2] = max(start_local[2], last_input.ready_time)
            last_input.ft_l = last_input.start_time[2] + last_input.speed_on_core[2]

            start_local[2] = last_input.ft_l


        """ ready1 and ready 2 updated"""
        corresponding_seq = sequence[last_input.assignment]
        v_i_index = corresponding_seq.index(last_input.id)  # position of last_input in seq list
        if v_i_index != len(corresponding_seq) - 1:  # not be the last one
            next_node_id = corresponding_seq[v_i_index + 1]
        else:
            next_node_id = -1

        for each in tasks_:
            flag = 0
            for p in each.predecessors:
                """if node's parent scheduled before ready1 of node=0"""
                if p.check_kernel != "kernel_done":
                    flag += 1
                ready1[each.id - 1] = flag
            if each.id == next_node_id:
                ready2[each.id - 1] = 0

        for each in tasks_:
            """ adding node"""
            if (ready1[each.id - 1] == 0) and (ready2[each.id - 1] == 0) and (
                    each.check_kernel != "kernel_done") and (each not in stack):
                stack.append(each)

    for node in tasks_:
        node.check_kernel = None

    """ Kernel finished """

    """ print sch on cores"""
    tmp0 = 0
    for a in sequence:
        if tmp0 != 3:
            print('core {}'.format(tmp0+1), ' : ', [i for i in a])

        tmp0 += 1

    Time_now = calculate_time(tasks_)
    Energy_now = calculate_energy(tasks_, [1, 2, 4, 0.5])
    E_diff = energy_in - Energy_now
    T_diff = abs(Time_now - time_in)
    len_of_performance += 1

    print("Time and Energy: ", Time_now, Energy_now)
    if E_diff <= 1:
        break

for node in tasks_:
    if node.on_device == True:
        print("Task ID: {}, assigenment:{}, ready time: {}, start_time on core: {}".format(node.id, node.assignment + 1, node.ready_time, node.start_time[node.assignment]))

print("Final schedulability ")
tmp0 = 0
for a in sequence:
    if tmp0 !=3:
        print('core {}'.format(tmp0+1) , ' : ', [i for i in a])
    tmp0 +=1

T_final = calculate_time(tasks_)
E_final = calculate_energy(tasks_, [1, 2, 4, 0.5])
print("Final Time: {}, Final Energy: {}".format(Time_now, Energy_now))

Priority Order
Task:  1      Priority Score:  21.666666666666664
Task:  3      Priority Score:  14.666666666666666
Task:  2      Priority Score:  14.666666666666664
Task:  6      Priority Score:  14.0
Task:  4      Priority Score:  13.333333333333332
Task:  5      Priority Score:  11.333333333333332
Task:  7      Priority Score:  9.666666666666666
Task:  8      Priority Score:  8.333333333333332
Task:  9      Priority Score:  7.666666666666666
Task:  10      Priority Score:  4.333333333333333
ID of Task ->1, sch on core ->3, ready time -> 0, start_time on core -> 0  sch line:  [0, 0, 5]
ID of Task ->3, sch on core ->3, ready time -> 5, start_time on core -> 5  sch line:  [0, 0, 9]
ID of Task ->2, sch on core ->2, ready time -> 5, start_time on core -> 5  sch line:  [0, 11, 9]
ID of Task ->6, sch on core ->1, ready time -> 5, start_time on core -> 5  sch line:  [12, 11, 9]
ID of Task ->4, sch on core ->3, ready time -> 5, start_time on core -> 9  sch line:  [12, 11, 12]
ID of Task ->5, 